In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from quantum_jeopardy import QuantumJeopardy

# Quantum Jeopardy

In [3]:
game = QuantumJeopardy()

In [4]:
resource_id = ""
game.setup_backend(resource_id)

['ionq.qpu', 'ionq.simulator']


In [6]:
game.play_round()

     ┌───┐     ┌────────────┐┌────────────┐
q_0: ┤ H ├──■──┤ Rx(2.8093) ├┤ Rz(4.2585) ├
     └───┘┌─┴─┐├────────────┤├────────────┤
q_1: ─────┤ X ├┤ Rx(2.8189) ├┤ Rz(4.8874) ├
          └───┘└────────────┘└────────────┘
c: 2/══════════════════════════════════════
                                           


What gate would you like to use? (Type None if no gate):  x
Which qubit would you like to apply the gate to?:  0
What gate would you like to use? (Type None if no gate):  x
Which qubit would you like to apply the gate to?:  1
What gate would you like to use? (Type None if no gate):  y
Which qubit would you like to apply the gate to?:  1


     ┌───┐     ┌────────────┐┌────────────┐┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤ Rx(2.8093) ├┤ Rz(4.2585) ├┤ X ├─────┤M├───
     └───┘┌─┴─┐├────────────┤├────────────┤├───┤┌───┐└╥┘┌─┐
q_1: ─────┤ X ├┤ Rx(2.8189) ├┤ Rz(4.8874) ├┤ X ├┤ Y ├─╫─┤M├
          └───┘└────────────┘└────────────┘└───┘└───┘ ║ └╥┘
c: 2/═════════════════════════════════════════════════╩══╩═
                                                      0  1 
Job id 5c9277b8-819e-11ec-b717-367dda17a95d
Job Status: job has successfully run
{'00': 5, '01': 44, '10': 44, '11': 5}


2

In [7]:
game.play_round(make_random = False)

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
c: 2/══════════
               


What gate would you like to use? (Type None if no gate):  x
Which qubit would you like to apply the gate to?:  1
What gate would you like to use? (Type None if no gate):  t
Which qubit would you like to apply the gate to?:  0
What gate would you like to use? (Type None if no gate):  y
Which qubit would you like to apply the gate to?:  1


     ┌───┐     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤ T ├─────┤M├───
     └───┘┌─┴─┐├───┤┌───┐└╥┘┌─┐
q_1: ─────┤ X ├┤ X ├┤ Y ├─╫─┤M├
          └───┘└───┘└───┘ ║ └╥┘
c: 2/═════════════════════╩══╩═
                          0  1 
Job id 8168af8a-819e-11ec-b717-367dda17a95d
Job Status: job has successfully run
{'00': 50, '01': 0, '10': 0, '11': 50}


0